# The Complete LangChain and LLM Guide
## Module 4: LangChain Prompt Templates

In [1]:
import os
import openai
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
from typing import List, Dict

#### Using the OpenAI function from openai library

In [2]:
from openai import OpenAI

In [3]:
dotenv_path = Path(".../.env")
def get_openai_key():
    load_dotenv(dotenv_path=dotenv_path)
    key = os.getenv("OPENAI_API_KEY")
    if not key:
        raise EnvironmentError("OPENAI_API_KEY is missing.")
    return key

In [4]:
openai_api_key = get_openai_key()

#### OpenAI API Prices    

| Model               | price per 1M Input Tokens | price per 1M Output Tokens | Characteristics / Notes  |    
---------------------|------------------------|--------------------------|----------------------------------------------------------------------------------------|    
| **GPT-5 nano**      | 0.05                   | 0.40                     | Fastest, cheapest GPT-5 variant; ideal for summarization/classification  |
| **GPT-4o mini**     | 0.15                   | 0.60                     | Multimodal (text, image, audio); cheaper than gpt-3.5; smaller and faster variant  |    
| **GPT-5 mini**      | 0.25                   | 2.00                     | Mid-tier GPT-5 variant, balanced speed and cost  |
| **gpt-3.5-turbo**   | 0.50                   | 1.50                     | Highly cost-effective for general chat and coding; 16k context window  |     
| **GPT-5 (full)**    | 1.25                   | 10.00                    | Flagship GPT-5: high performance, large 400k+ context window |
| **GPT-4o (full)**   | 2.50                   | 10.00                    | Powerful multimodal model (vision, speech, translation); high performance  |
| **GPT-4.5**         | 75.00                  | 150.00                   | Very large model, creative insights, training via unsupervised learning; expensive |
| **o1-pro**          | 150.00                 | 600.00                   | Top-tier “reasoning” model; highest cost yet, powerful STEM capability |


In [5]:
llm_model = "gpt-5-nano"

In [6]:
llm = OpenAI(api_key=openai_api_key)

In [7]:
type(llm)

openai.OpenAI

In [8]:
customer_review = """
    I can't believe I wasted my time and money on this pathetic excuse for a service. 
    This is a scam, and anyone who buys from you is an idiot. 
    You people are a joke.
"""

### Completions with OpenAI

In [9]:
def get_completion(client, model, messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1
        )
    return response.choices[0].message.content

In [10]:
template = f"""
    Rewrite the customer review message in a polite tone, and then please translate the new review message into Portuguese.
    Review: {customer_review}
"""

In [11]:
template

"\n    Rewrite the customer review message in a polite tone, and then please translate the new review message into Portuguese.\n    Review: \n    I can't believe I wasted my time and money on this pathetic excuse for a service. \n    This is a scam, and anyone who buys from you is an idiot. \n    You people are a joke.\n\n"

In [12]:
# create the message list 
messages = [{"role":"user", "content":template}]
messages

[{'role': 'user',
  'content': "\n    Rewrite the customer review message in a polite tone, and then please translate the new review message into Portuguese.\n    Review: \n    I can't believe I wasted my time and money on this pathetic excuse for a service. \n    This is a scam, and anyone who buys from you is an idiot. \n    You people are a joke.\n\n"}]

In [13]:
response = get_completion(llm, llm_model, messages)
print(response)

Polite rewrite (English):
Regrettably, my experience with this service did not meet my expectations, and I feel that my time and money were not well spent. I have concerns about the transparency and value of the offering. I cannot recommend this service to others, and I hope you will consider taking steps to improve going forward.

Portuguese translation:
Infelizmente, minha experiência com este serviço não atendeu às minhas expectativas e sinto que meu tempo e meu dinheiro não foram bem gastos. Tenho preocupações sobre a transparência e o valor da oferta. Não posso recomendar este serviço a outras pessoas e espero que vocês considerem tomar medidas para melhorar daqui para frente.


### Completions with LangChain and Prompt Templates - ChatAPI

#### Explanation of ChatPromptTemplate.from_template (from Gemini)

#### `ChatPromptTemplate.from_template`

`ChatPromptTemplate.from_template` is a convenient way to create a `ChatPromptTemplate` in LangChain from a simple string template. It automatically handles the conversion of a single string into a list of messages, assuming the user's input is a single message.

The main benefit of using `from_template` is its simplicity for common use cases. You don't need to manually define the `system` and `human` messages as separate components. The method takes a single string, and it automatically creates a `HumanMessagePromptTemplate` from it.

Here's a breakdown of how it works and a simple example:

**How it works:**

1.  **Input:** You provide a single string that acts as a template for the human message. You can include placeholders (e.g., `{input_variable}`) that will be filled in later.

2.  **Automatic Conversion:** LangChain's `from_template` method takes this string and creates a `HumanMessagePromptTemplate` object. This object represents a message from the user.

3.  **Result:** The final output is a `ChatPromptTemplate` with a single message template inside it. When you format this template with your input variables, it produces a list containing a single `HumanMessage` object.

**Example:**

Let's say you want to create a simple prompt that asks a user for a company name and then suggests a catchy slogan.

```python
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

# Define the template string
template = "What is a good slogan for a company named {company_name}?"

# Create the ChatPromptTemplate using from_template
prompt = ChatPromptTemplate.from_template(template)

# Now, let's use the prompt
input_variables = {"company_name": "Acme Inc."}
formatted_messages = prompt.format_messages(**input_variables)

# The result is a list of Chat messages
print(formatted_messages)
print(type(formatted_messages[0]))
```

**Output:**

```
[HumanMessage(content='What is a good slogan for a company named Acme Inc.')]
<class 'langchain_core.messages.human.HumanMessage'>
```

**Key Takeaways:**

  * **Simplicity:** Best for simple, single-message prompts.
  * **Automatic `HumanMessage`:** It automatically wraps your template string in a `HumanMessagePromptTemplate`.
  * **Alternative to `from_messages`:** It's a quick alternative to `ChatPromptTemplate.from_messages` **when you only need a single user message**.
  * **Limitations:** For more complex prompts that require multiple message types (e.g., `SystemMessage`, `AIMessage`, or a series of `HumanMessage`s), you should use `ChatPromptTemplate.from_messages`.

**When to use `from_template`:**

  * You're building a simple chatbot or tool that takes a single user query.
  * You want a quick and easy way to create a `ChatPromptTemplate` without a lot of boilerplate code.
  * Your prompt only consists of a single `HumanMessage`.

**In summary**, `ChatPromptTemplate.from_template` is your go-to function for creating a simple, single-message prompt in LangChain.

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [15]:
chat_llm = ChatOpenAI(api_key=openai_api_key, model = llm_model)

In [16]:
# Define the template string
template = """
    Rewrite the customer review message in a polite tone, and then please translate the new review message into Italian.
    Review: {customer_review}
"""

In [17]:
# Create the ChatPromptTemplate using from_template
prompt = ChatPromptTemplate.from_template(template)

In [18]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [19]:
# Create a runnable sequence by chaining the prompt and the model
chain = prompt | chat_llm

In [20]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [21]:
# Define input variables
input_variables = {"customer_review": customer_review}
input_variables

{'customer_review': "\n    I can't believe I wasted my time and money on this pathetic excuse for a service. \n    This is a scam, and anyone who buys from you is an idiot. \n    You people are a joke.\n"}

In [22]:
response = chain.invoke(input_variables)

In [23]:
type(response)

langchain_core.messages.ai.AIMessage

In [24]:
# The result is a list of Chat messages
print(response)
print("---------------------")
print(response.content)

content="- Polite rewrite (English):\nI am disappointed with the service I received and feel that it did not meet my expectations or provide good value for the money I spent. I have concerns about transparency and reliability, and I would not currently recommend this company to others.\n\n- Italian translation:\nMi sono sentito deluso dal servizio che ho ricevuto e ritengo che non abbia soddisfatto le mie aspettative né offerto un buon valore per i soldi che ho speso. Ho dubbi sulla trasparenza e sull'affidabilità, e al momento non consiglierei questa azienda ad altri." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2754, 'prompt_tokens': 76, 'total_tokens': 2830, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 2624, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 